**ArASL
Arabic Arm Sign Language Image Classification**

**1- Import needed libraries**<br>
**2- Load Data**<br>
**3- Define Functions**<br>
**4- Preparing Data**<br>
**5- Model Definition**<br>
**6- Model Training**<br>
**7- Model Evaluation**<br>
**8- Reports**<br>
**9- Model Prediction**<br><br>

**1- Import needed libraries**

In [1]:
import seaborn as sns
import arabic_reshaper
from random import randint
import matplotlib.pyplot as plt
from keras import utils as Utils
from sklearn.utils import shuffle
from keras import models as Models
from keras import layers as Layers
from keras.preprocessing import image
from bidi.algorithm import get_display
import matplotlib.gridspec as gridspec
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from keras.layers import Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
import os, numpy as np, pandas as pd, random, sklearn, keras, os, tensorflow as tf
from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization

In [ ]:
# global variables
Language = "Ar"
ImageClassMapping_path = "ArASL_Database_54K/Labels/ImagesClassPath.csv"
ClassLabels_path = "ArASL_Database_54K/Labels/ClassLabels.xlsx"
ImagesRoot_path = "ArASL_Database_54K"

# ModelFileName ='Model_255.h5'

**2- Load Data**


In [ ]:
# load 54k image path mapping
df_ImageClassPath = pd.read_csv(ImageClassMapping_path)

In [ ]:
# load Class Labels
df_Classes = pd.read_excel(ClassLabels_path)

In [ ]:
# df_ImageClassPath.groupby("ClassId").size().describe()

In [ ]:
def SplitData(predictions,testsize):
    
    min = df_ImageClassPath.groupby("ClassId").size().min()
    print('{0} Samples per Class'.format(min))
    
    # empty dataframes with same column difinition
    df_TrainingSet = df_ImageClassPath[0:0].copy()
    df_TestSet = df_ImageClassPath[0:0].copy()
    df_PredSet = df_ImageClassPath[0:0].copy()

    # Create the sets by loop thru classes and append
    for index,row in df_Classes.iterrows():
        # make sure all class are same size 
        df_FullSet = df_ImageClassPath[df_ImageClassPath['ClassId'] == row['ClassId']].sample(min,random_state= 42)
        
#         df_FullSet = df_ImageClassPath[df_ImageClassPath['ClassId'] == row['ClassId']]
        
        df_PredSet = pd.concat([df_PredSet , df_FullSet.sample(n=predictions, random_state=1)])
        df_FullSet = pd.merge(df_FullSet,df_PredSet, indicator=True, 
                              how='left').query('_merge=="left_only"').drop('_merge', axis=1)
        
        trainingSet, testSet = train_test_split(df_FullSet, test_size= testsize)        
        
        df_TrainingSet =  pd.concat([df_TrainingSet,trainingSet])
        df_TestSet = pd.concat([df_TestSet,testSet])
    
    return df_TrainingSet,df_TestSet,df_PredSet

In [ ]:
# def SplitData(predictions,testsize):
    
#     min = df_ImageClassPath.groupby("ClassId").size().min()
#     print('{0} Samples per Class'.format(min))
    
#     # empty dataframes with same column difinition
#     df_TrainingSet = df_ImageClassPath[0:0].copy()
#     df_TestSet = df_ImageClassPath[0:0].copy()
#     df_PredSet = df_ImageClassPath[0:0].copy()

#     # Create the sets by loop thru classes and append
#     for index,row in df_Classes.iterrows():
#         # make sure all class are same size 
#         df_FullSet = df_ImageClassPath[df_ImageClassPath['ClassId'] == row['ClassId']].sample(min,random_state= 42)
        
# #         df_FullSet = df_ImageClassPath[df_ImageClassPath['ClassId'] == row['ClassId']]
        
#         df_PredSet = df_PredSet.append(df_FullSet.sample(n=predictions, random_state=1))
#         df_FullSet = pd.merge(df_FullSet,df_PredSet, indicator=True, 
#                               how='left').query('_merge=="left_only"').drop('_merge', axis=1)
        
#         trainingSet, testSet = train_test_split(df_FullSet, test_size= testsize)        
        
#         df_TrainingSet = df_TrainingSet.append(trainingSet)
#         df_TestSet = df_TestSet.append(testSet)
    
#     return df_TrainingSet,df_TestSet,df_PredSet

In [ ]:
# retrive class Label (Arabic or English) using class id 
def get_classlabel(class_code,lang= 'Ar'):
    if lang== 'Ar':
        text_to_be_reshaped = df_Classes.loc[df_Classes['ClassId'] == class_code, 
                                             'ClassAr'].values[0]
        reshaped_text = arabic_reshaper.reshape(text_to_be_reshaped)
        return get_display(reshaped_text)
    elif lang== 'En':
        return df_Classes.loc[df_Classes['ClassId'] == class_code, 'Class'].values[0]

In [ ]:
def getDataSet(setType,isDL): # 'Training' for Training dataset , 'Testing' for Testing data set
    imgs = []
    lbls = []
    df = pd.DataFrame(None)
    
    if setType =='Training':
        df = dtTraining.copy()
    elif setType=='Test':
        df = dtTest.copy()
    elif setType=='Prediction':
        df = dtPred.copy()

    for index,row in df.iterrows():
        lbls.append(row['ClassId'])
        try:
            imageFilePath = os.path.join(ImagesRoot_path, row['ImagePath'])
            img = image.load_img(imageFilePath, target_size=(64,64,1), 
                                 color_mode = "grayscale")
            img = image.img_to_array(img) # to array
            img = img/255 # Normalize
            if isDL == False:
                img = img.flatten() # for knn_classifier Model
            imgs.append(img)

        except Exception as e:
            print(e)
            
    shuffle(imgs,lbls,random_state=255) #Shuffle the dataset

    imgs = np.array(imgs)
    lbls = np.array(lbls)
    if isDL ==True:
        lbls = to_categorical(lbls) # for keras CNN Model
    return imgs, lbls

In [ ]:
def display_prediction(col_size, row_size,XPred,yPred): 
    img_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(XPred[img_index][:,:,0], cmap='gray')
            ax[row][col].set_title("({}) {}".format(yPred[img_index],get_classlabel(yPred[img_index],'Ar')))
            ax[row][col].set_xticks([])
            ax[row][col].set_yticks([])
            img_index += 1

**4- Preparing Data**

In [ ]:
dtTraining, dtTest,dtPred = SplitData(3,0.3)

In [ ]:
X_train,y_train = getDataSet('Training',True)
X_valid,y_valid = getDataSet('Test',True)
X_pred,_ = getDataSet('Prediction',True)

In [ ]:
print("Shape of Train Images:{} , Train Labels: {}".format(X_train.shape,y_train.shape))
print("Shape of Test Images:{} , Test Labels: {}".format(X_valid.shape,y_valid.shape))
print("Shape of Prediction Images:{} , Prediction Labels: {}".format(X_pred.shape,"?"))

 You can uncomment from Heading 5 to 8 for training Purpose only otherwise while using this to your code Dont uncomment it.

**5. Keras Convolutional Neural Network (CNN)**

In [ ]:
# model = Models.Sequential()

# model.add(Layers.Conv2D(64, kernel_size=(3, 3),activation='relu',input_shape=(64,64,1)))

# model.add(Layers.Conv2D(32, (3, 3), activation='relu'))
# model.add(Layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(Layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(Layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(Layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(Layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(Layers.Dropout(0.25))
# model.add(Layers.Flatten())
# model.add(Layers.Dense(128, activation='relu'))
# model.add(Layers.Dropout(0.5))
# model.add(Layers.Dense(32, activation='softmax'))

# model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# model.summary()
# # Utils.plot_model(model,to_file='model.png',show_shapes=True, show_layer_names=True, dpi=80)


**7- Model Evaluation**

In [ ]:
# plt.plot(trained.history['accuracy'])
# plt.plot(trained.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

# plt.plot(trained.history['loss'])
# plt.plot(trained.history['val_loss'])
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

In [ ]:
# print("on Validation data")
# pred1=model.evaluate(X_valid,y_valid)
# print("accuaracy", str(pred1[1]*100))
# print("Total loss",str(pred1[0]*100))

In [ ]:
# Previous Model
# ModelFileName ='Model_255.h5'
# 387/387 [==============================] - 1s 4ms/step - loss: 0.2808 - accuracy: 0.9470
# accuaracy 94.7028398513794
# Total loss 28.077486157417297

In [ ]:
# New Model
# ModelFileName ='Model_256.h5'
# 387/387 [==============================] - 1s 3ms/step - loss: 0.1163 - accuracy: 0.9755
# accuaracy 97.55329489707947
# Total loss 11.626577377319336

**8- Reports**

In [ ]:
# Y_prediction = model.predict(X_valid)
# # Convert predictions classes to one hot vectors 
# Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
# # Convert validation observations to one hot vectors
# Y_true = np.argmax(y_valid,axis = 1) 
# # compute the confusion matrix
# confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 

# plt.figure(figsize=(10,8))
# plt.title('Validation confusion_matrix', fontsize = 16) 
# sns.heatmap(confusion_mtx, annot=True, fmt="d");


In [2]:
restored_model = load_model('model_256.h5')
# restored_model.summary()

In [3]:
def fun(img):
    img = (image.img_to_array(img)/255.0).reshape(1,64,64,1)   
    pred = restored_model.predict(img)
    cnn_pred = np.argmax(pred,axis = 1)
#     word = get_classlabel(cnn_pred[0],'Ar')
    return cnn_pred

**9- Model Prediction**

In [4]:
ImagePath = "ArASL_Database_54K/ArASL_Database_54K/ain/AIN (1).jpg"
img = image.load_img(ImagePath, target_size=(64,64,1),color_mode = "grayscale") 

In [5]:
fun(img)

array([0], dtype=int64)

In [ ]:
# #Un-comment it when you want to watch your input and label on it.
# cnn_Y_pred = np.argmax(fun(img),axis = 1)
# fig,ax = plt.subplots(figsize=(2*2.5,2*1.5))
# ax.imshow(X_pred[0][:,:,0], cmap='gray')
# ax.set_title("({}) {}".format(cnn_Y_pred[0],get_classlabel(cnn_Y_pred[0],'Ar')))

In [ ]:
# cnn_pred = np.argmax(fun(img),axis = 1)
# get_classlabel(cnn_pred[0],'Ar')

In [6]:
import glob
import cv2
for img in glob.glob(""):
    img = cv2.imread(img)
#     cv_img.append(n)
    print(n.shape)

(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64,

(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64,

(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
(64,